In [ ]:
from copy import copy, deepcopy
import pandas as pd
import numpy as np
import sys
sys.append('../input/coderepro')
from method4memreducing import get_dcode_mapping, get_dcode_and_ldcode
from select_samples import extract_tran4feat, extract_tran4ibcf
from sample_method import generate_samples4finedrank_direct, generate_samples4infer, generate_samples_feature4finedrank
from select_features import get_mv_for_features, get_importance_from_tree
from recall_phrase.merge_multi_method import get_recall_by_ibcf_and_majority
from feature_engineer.article_features import extract_arr_features
from feature_engineer.customer_features import extract_apk_features
from feature_engineer.interacting_features import extract_historical_purchased_items
from model import train_tree_model
from recall_phrase.item_base_cf.recall import get_customer_purchasing_items
from method4infer import pred

In [ ]:
transactions_train_csv = '../input/h-and-m-personalized-fashion-recommendations/transactions_train.csv'
transactions = pd.read_csv(transactions_train_csv, dtype={'article_id': str})
transactions['customer_id'] = transactions['customer_id'].apply(lambda x: int(x[-16:],16) ).astype('int64')
transactions['article_id'] = transactions['article_id'].astype('int32')
transactions['t_dat_dateform'] = pd.to_datetime(transactions['t_dat'])


article_csv = '../input/h-and-m-personalized-fashion-recommendations/articles.csv'
article = pd.read_csv(article_csv)
article['article_id'] = article['article_id'].astype('int32')


customer_csv = '../input/h-and-m-personalized-fashion-recommendations/customers.csv'
customer = pd.read_csv(customer_csv)
customer['customer_id'] = customer['customer_id'].apply(lambda x: int(x[-16:],16) ).astype('int64')

In [ ]:
d2c_mapping, c2d_mapping = get_dcode_mapping(transactions['t_dat_dateform'])
transactions = get_dcode_and_ldcode(transactions, d2c_mapping)

'''
get transaction for feature, model and recall for training model part
'''
# ld4feat_trn_cv = '2020-09-08'
# ld4ibcf_trn_cv = '2020-09-08'
# tran4feat_trn_cv, tran4model_trn_cv = extract_tran4feat(
#         transactions,
#         ld4feat_trn_cv,
#         d2c_mapping,
#         tran4feat_sessions=28,
#         is_tran4model=True,
# )
# tran4ibcf_trn_cv = extract_tran4ibcf(
#     transactions,
#     ld4ibcf_trn_cv,
#     d2c_mapping,
#     tran4ibcf_sessions=21,
# )


ld4feat_trn_lb = '2020-09-15'
ld4ibcf_trn_lb = '2020-09-15'
tran4feat_trn_lb, tran4model_trn_lb = extract_tran4feat(
        transactions,
        ld4feat_trn_lb,
        d2c_mapping,
        tran4feat_sessions=28,
        is_tran4model=True,
)
tran4ibcf_trn_lb = extract_tran4ibcf(
    transactions,
    ld4ibcf_trn_lb,
    d2c_mapping,
    tran4ibcf_sessions=21,
)


'''
get transaction for feature, model and recall for inferring part
'''
# ld4feat_dev_cv = '2020-09-15'
# tran4feat_dev_cv, tran4model_dev_cv = extract_tran4feat(
#         transactions,
#         ld4feat_dev_cv,
#         d2c_mapping,
#         tran4feat_sessions=28,
#         is_tran4model=True,
# )
ld4feat_inf_lb = '2020-09-22'
tran4feat_inf_lb, _ = extract_tran4feat(
        transactions,
        ld4feat_inf_lb,
        d2c_mapping,
        tran4feat_sessions=28,
        is_tran4model=False,
)

In [ ]:
'''
get recall items for training and dev in cv
'''
# W4ibcf_trn_cv, cpi4recall_trn_cv = get_recall_by_ibcf_and_majority(tran4ibcf_trn_cv)
# W4ibcf_dev_cv, cpi4recall_dev_cv = get_recall_by_ibcf_and_majority(tran4ibcf_dev_cv)


'''
get recall items for training and inference in lb
'''
W4ibcf_trn_lb, cpi4recall_trn_lb = get_recall_by_ibcf_and_majority(tran4ibcf_trn_lb)
W4ibcf_inf_lb, cpi4recall_inf_lb = get_recall_by_ibcf_and_majority(tran4ibcf_inf_lb)


In [ ]:
'''
generate training samples
'''
# samples4finedrank_trn_cv = generate_samples4finedrank_direct(recall_and_purchased_trn_cv)
samples4finedrank_trn_lb = generate_samples4finedrank_direct(recall_and_purchased_trn_lb)


'''
generate inferring samples
'''
# purchased_items_dev_cv = get_customer_purchasing_items(tran4model_dev_cv)
# recall_and_purchased_dev_cv = combine_recall_and_purchasing_items_nw(
#     cpi4recall_dev_cv,
#     purchased_items_dev_cv,
#     majority_dev_cv
# )
recall_inf_lb = generate_samples4infer(customer, cpi4recall_inf_lb)


In [ ]:
# build feature dict for training set in local cross validation

# a_features_trn_cv = extract_arr_features(article, tran4feat_trn_cv, na_filler=-1) # it is a dataframe
# a_features_trn_cv = convert_df2dict(a_features_trn_cv, 'article_id')  # convert dataframe to dict

# c_features_trn_cv = extract_apk_features(customer, tran4feat_trn_cv, na_filler=-1)
# c_features_trn_cv = convert_df2dict(c_features_trn_cv, 'customer_id') 

# historical_purchasing_infor_trn_cv = extract_historical_purchased_items(tran4feat_trn_cv)


# build feature dict for dev set in local cross validation
# a_features_dev_cv = extract_arr_features(article, tran4feat_dev_cv, na_filler=-1)
# a_features_dev_cv = convert_df2dict(a_features_dev_cv, 'article_id')

# c_features_dev_cv = extract_apk_features(customer, tran4feat_dev_cv, na_filler=-1)
# c_features_dev_cv = convert_df2dict(c_features_dev_cv, 'customer_id') 

# historical_purchasing_infor_dev_cv = extract_historical_purchased_items(tran4feat_dev_cv)




# # build feature dict for training set in leading board
a_features_trn_lb = extract_arr_features(article, tran4feat_trn_lb, na_filler=-1)
a_features_trn_lb = convert_df2dict(a_features_trn_lb, 'article_id')

c_features_trn_lb = extract_apk_features(customer, tran4feat_trn_lb, na_filler=-1)
c_features_trn_lb = convert_df2dict(c_features_trn_lb, 'customer_id') 

historical_purchasing_infor_trn_lb = extract_historical_purchased_items(tran4feat_trn_lb)


# # build feature dict for inferring set in leading board
a_features_inf_lb = extract_arr_features(article, tran4feat_inf_lb, na_filler=-1)
a_features_inf_lb = convert_df2dict(a_features_inf_lb, 'article_id')

c_features_inf_lb = extract_apk_features(customer, tran4feat_inf_lb, na_filler=-1)
c_features_inf_lb = convert_df2dict(c_features_inf_lb, 'customer_id') 

historical_purchasing_infor_inf_lb = extract_historical_purchased_items(tran4feat_inf_lb)


In [ ]:
samples_feature4finedrank_trn_lb = generate_samples_feature4finedrank(
    samples4finedrank_trn_lb,
    a_features_trn_lb,
    c_features_trn_lb,
    historical_purchasing_infor_trn_lb)


# drop features are not in numeric dataform
samples_feature4finedrank_trn_lb = samples_feature4finedrank_trn_lb.drop(
    ['club_member_status', 'fashion_news_frequency', 'postal_code'], axis=1
)

In [ ]:
'''
select features
'''

# by mv test
# mv_for_features = get_mv_for_features(
#     data=samples_feature4finedrank_trn_cv,
#     label='label' 
# )

mv_for_features = get_mv_for_features(
    data=samples_feature4finedrank_trn_lb,
    label='label' 
)

selected_features_from_mv = []
for feature, info in mv_for_features.items():
    if info['p-value'][1] <= 0.05:
        selected_features_from_mv.append(feature)
        

# by tree
# _, feature_importance_from_lgb , features_and_importance_from_lgb = get_importance_from_tree(
#     data=samples_feature4finedrank_trn_cv,
#     label='label',
#     mode='lgb',
# )

_, feature_importance_from_lgb , features_and_importance_from_lgb = get_importance_from_tree(
    data=samples_feature4finedrank_trn_lb,
    label='label',
    mode='lgb',
)


# merge
selected_features = list(set(selected_features_from_mv) | set(selected_features_from_lgb))
        

In [ ]:
'''
prediction
'''
# clf = train_tree_model(
#     data=samples_feature4finedrank_trn_cv,
#     selected_features=selected_features,
#     label='label',
#     mode='lgb'
# )

clf = train_tree_model(
    data=samples_feature4finedrank_trn_lb,
    selected_features=selected_features,
    label='label',
    mode='lgb'
)

In [ ]:
'''
inference
'''

# prediction_dev_cv = pred(
#     recall_and_purchased_dev_cv,
#     clf,
#     a_features_dev_cv,
#     c_features_dev_cv,
#     historical_purchasing_infor_dev_cv,
#     selected_features,
# )


# prediction_dev_cv


prediction_inf_lb = pred(
    recall_inf_lb,
    clf,
    a_features_inf_lb,
    c_features_inf_lb,
    historical_purchasing_infor_inf_lb,
    selected_features,
)

In [ ]:
'''
merge and submit
'''
sample_submission = pd.read_csv(
    "../input/h-and-m-personalized-fashion-recommendations/sample_submission.csv",   
)
sample_submission['customer_id2'] = sample_submission['customer_id'].apply(lambda x: int(x[-16:],16) ).astype('int64')

prediction4submission = deepcopy(prediction_dev_cv)
prediction4submission = deepcopy(prediction_inf_lb)
prediction4submission = prediction4submission.rename(
    columns={'customer_id': 'customer_id2'}
)
sample_submission = pd.merge(
    sample_submission,
    prediction4submission,
    how='left',
    on='customer_id2',
)

# 'identity' is helping field, used to drop those without prediction
sample_submission['identity'] = sample_submission['recommend'].apply(
    lambda x: 1 if isinstance(x, list) else 0
)
sample_submission = sample_submission.query('identity == 1').reset_index(drop=True)
sample_submission = sample_submission.drop(columns=['customer_id2', 'identity'])

sample_submission['recommend'] = sample_submission['recommend'].apply(
    lambda x: ['0' + str(item) for item in x]
)
sample_submission = sample_submission[['customer_id', 'recommend']]
sample_submission.to_csv('submission.csv',index=False)